# Word2Vec Implementation from Scratch

This notebook demonstrates how to implement the Word2Vec algorithm from scratch using PyTorch. We'll use the first Harry Potter book as our corpus to train word embeddings.


## 1. Setting Up the Environment

First, we need to import the necessary libraries:
- `torch` and `torch.nn` for tensor operations and neural network functionality
- `string` for string manipulations (removing punctuation)


In [1]:
import torch
import torch.nn as nn
import string


## 2. Getting the Text Data

We'll download the first Harry Potter book to use as our corpus.

In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2025-03-18 05:22:14--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  2.11MB/s    in 0.2s    

2025-03-18 05:22:15 (2.11 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



## 3. Text Preprocessing

Before we can use the text data, we need to preprocess it:
- Remove punctuation
- Convert text to lowercase
- Split text into tokens (words)

This function will help us clean and tokenize the text.

In [3]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


## 4. Loading and Formatting the Text

Now we'll load the text file, replace some special characters, and split the text into sentences.


In [4]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
list_of_sentences = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')

Let's tokenize the text using our preprocessing function `make_tokenized_corpus`:

In [5]:
# Corpus is a list of list of strings (words)

for sentence in list_of_sentences[:10]:
  print(sentence)

Harry Potter and the Sorcerer's Stone   CHAPTER ONE  THE BOY WHO LIVED  mr and mrs Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense
 mr Dursley was the director of a firm called Grunnings, which made drills
He was a big, beefy man with hardly any neck, although he did have a very large mustache
mrs Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors
The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere
 The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it
They didn't think they could bear it if anyone found out about the Potters
mrs Potter was

In [6]:
corpus = make_tokenized_corpus(list_of_sentences)

type(corpus), type(corpus[0]), type(corpus[0][0])

(list, list, str)

## 5. Creating Context Word Pairs

A key concept in Word2Vec is learning from context. We need to create pairs of words that appear near each other in the text. We'll use a sliding window approach to create these pairs.

For example, with the window size of 2, for the word "to" in the sentence "they were the last people youd expect to be involved...", we would create pairs with:
- ("to", "expect")
- ("to", "be")
- ("to", "involved")
- ("to", "in")

These pairs will be our training data.

In [7]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_pairs = []
window_size = 2

for sample_sentence in tqdm(corpus):
  for cur_idx, center_word in enumerate(sample_sentence):
    window_begin = max(cur_idx - window_size, 0)
    window_end = min(cur_idx + window_size + 1, len(sample_sentence))
    # for context_word in sample_sentence[window_begin:window_end]:
    #   # if center_word == context_word: continue
    #   word_pairs.append( (center_word, context_word))
    for j in range(window_begin, window_end):
      if cur_idx == j: continue
      word_pairs.append( (center_word, sample_sentence[j]))

print(f"\nLength of word_pairs is {len(word_pairs)}")
print(f"First 5 example of word_pairs is {word_pairs[:5]}")

100%|██████████| 4682/4682 [00:00<00:00, 5503.03it/s]


Length of word_pairs is 282372
First 5 example of word_pairs is [('harry', 'potter'), ('harry', 'and'), ('potter', 'harry'), ('potter', 'and'), ('potter', 'the')]


## 6. Building the Vocabulary

To work with word vectors, we need to create a vocabulary that maps each unique word to an index. We'll also filter out rare words that appear less than a certain number of times in the corpus.

### 6.1 Collecting All Words

First, let's collect all words in our corpus:


In [8]:
# we have to make vocabulary
sentence = corpus[0]
entire_words = []

for sentence in corpus:
  for word in sentence:
    entire_words.append(word)

len(entire_words)

77597


### 6.2 Finding Unique Words

Now, let's find the unique words in our corpus:

In [9]:
# we have to get the "unique" item among total words

vocab_set = set(entire_words)
len(vocab_set)

unique_words = list(vocab_set)
len(unique_words)

6038

### 6.3 Converting to a List and Sorting

We'll convert the set of unique words to a sorted list:

In [10]:
# vocab_set[0] # set is not subscriptable because it has no order

unique_words = sorted(list(unique_words))
unique_words[0]

'\the'

### 6.4 Filtering by Frequency

Now, let's filter out rare words that occur less than a specified number of times:
- We can use the `Counter` class from the `collections` module to count the frequency of each word in the corpus.
- Caution on `alist.sort()` will return `None`.

In [11]:
# how can we filter the vocab by its frequency?
filtered_vocab = None
# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)

from collections import Counter
word_counter = Counter(entire_words)
word_counter.most_common(10)
word_counter['harry']

threshold = 5
filtered_vocab = []
for key, value in word_counter.items():
  if value > threshold:
    filtered_vocab.append(key)

filtered_vocab.sort()
filtered_vocab[:10]

['a',
 'able',
 'abou',
 'about',
 'above',
 'across',
 'added',
 'afford',
 'afraid',
 'after']

## 7. Filtering Word Pairs

Now that we have our filtered vocabulary, we need to filter our word pairs to only include words that are in our vocabulary:

In [12]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_word_pairs = []
vocab_set = set(filtered_vocab)

for pair in tqdm(word_pairs):
  a, b = pair
  if a in filtered_vocab and b in filtered_vocab:
    filtered_word_pairs.append(pair)

100%|██████████| 282372/282372 [00:11<00:00, 25466.78it/s]


In [20]:
# implement same algorithm with list comprehension

filtered_word_pairs = [pair for pair in word_pairs if pair[0] in vocab_set and pair[1] in vocab_set]

In [14]:
len(filtered_word_pairs), len(word_pairs)

(226846, 282372)

## 8. Converting Words to Indices

For efficiency, we'll convert our words to indices according to their position in our vocabulary:

In [15]:
# convert word into index of vocab
# filtered_vocab.index('happily')
filtered_vocab.index('harry')

527

This is inefficient because `list.index()` has to scan the list every time. Let's use a dictionary for faster lookups:

In [17]:
# we can make it faster
# use dictionary to find the index of string
word2idx = dict()
for idx, word in enumerate(filtered_vocab):
  word2idx[word] = idx

word2idx['harry']

527

Now, let's convert our word pairs to index pairs more efficiently:

In [25]:
index_pairs = [(word2idx[pair[0]], word2idx[pair[1]]) for pair in filtered_word_pairs]
index_pairs[0]

(527, 953)

In [26]:
# Why we don't need idx2tok?

filtered_vocab[527]

'harry'

## 9. Creating Initial Word Vectors

Now we'll create random vectors for each word in our vocabulary. These vectors will be adjusted during training:
- We can use `torch.randn` to create random vectors that follow normal distribution.

In [42]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 100
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 10
word_vectors

tensor([[-0.0726, -0.1422, -0.1386,  ...,  0.1413,  0.0136,  0.0928],
        [ 0.0387,  0.0684, -0.0188,  ...,  0.0316,  0.0317, -0.1438],
        [-0.0268, -0.0022, -0.0271,  ...,  0.0574,  0.0139,  0.1331],
        ...,
        [ 0.0339,  0.0197, -0.1009,  ...,  0.0948, -0.0939, -0.1432],
        [ 0.0111, -0.0825,  0.0333,  ..., -0.0428,  0.0658, -0.0670],
        [-0.1141, -0.0913,  0.0875,  ..., -0.0803,  0.1407,  0.0963]])

In [41]:
# what is the vector for harry?
word_vectors[word2idx['harry']]

tensor([ 0.1187, -0.1411, -0.0562, -0.0851,  0.0260,  0.1393, -0.0842, -0.0380,
         0.0117,  0.1649,  0.0039,  0.0540, -0.0452, -0.0119,  0.0271, -0.1264,
        -0.0613, -0.0285,  0.1139, -0.0633, -0.0860, -0.1596, -0.0210, -0.1657,
         0.0979,  0.0501, -0.0266, -0.0172,  0.1560, -0.2785, -0.0872, -0.0325,
        -0.1014,  0.0164, -0.0384,  0.0729, -0.0312, -0.1175, -0.1621,  0.0580,
         0.0072,  0.0178, -0.0772, -0.1279, -0.1002, -0.0670,  0.0744, -0.1892,
        -0.2057, -0.0350, -0.0114, -0.0350,  0.0042,  0.1155, -0.0968, -0.1576,
         0.0171, -0.0546,  0.0132,  0.0462,  0.1590,  0.0126, -0.0596,  0.0968,
        -0.0713,  0.0703, -0.0761,  0.0783,  0.0505, -0.0598,  0.0283,  0.0273,
        -0.1183, -0.0937, -0.0943, -0.0072,  0.0384,  0.0667,  0.0424, -0.0901,
         0.0037, -0.0839,  0.0018, -0.0168, -0.0254,  0.0090, -0.0418,  0.1053,
        -0.0848,  0.0636, -0.0892,  0.0880, -0.0231,  0.1676, -0.0815, -0.2471,
        -0.0098,  0.0412,  0.3541,  0.03

## 10. Understanding Word Relationships with Dot Products

The core of Word2Vec is using dot products to measure relationships between words. Let's explore this concept:

In [56]:
torch.set_printoptions(sci_mode=False) # Do this to avoid scientific notation

## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$

Let's calculate the dot product between "harry" and "potter":


In [57]:
# calculate P(potter|harry)
harry = word_vectors[word2idx['harry']]
potter = word_vectors[word2idx['potter']]
dot_product_value_between_potter_harry = sum(harry * potter)
dot_product_value_between_potter_harry

tensor(-0.1254)

In [58]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
  w_idx = word2idx[word]
  w_vector = word_vectors[w_idx]
  word_dot_dict[word] = sum(harry * w_vector)
word_dot_dict

{'a': tensor(0.0147),
 'able': tensor(0.1449),
 'abou': tensor(-0.0150),
 'about': tensor(-0.0696),
 'above': tensor(-0.2321),
 'across': tensor(-0.0507),
 'added': tensor(0.1369),
 'afford': tensor(0.1165),
 'afraid': tensor(-0.1324),
 'after': tensor(0.2381),
 'afternoon': tensor(0.2201),
 'again': tensor(0.0086),
 'against': tensor(-0.0662),
 'ages': tensor(-0.0308),
 'ago': tensor(0.0238),
 'agreed': tensor(0.0588),
 'ah': tensor(0.2077),
 'ahead': tensor(0.0122),
 'air': tensor(-0.0275),
 'albus': tensor(-0.0412),
 'alive': tensor(0.0134),
 'all': tensor(-0.1455),
 'alley': tensor(0.1873),
 'allowed': tensor(-0.0368),
 'almost': tensor(0.0034),
 'alone': tensor(-0.0948),
 'along': tensor(0.0067),
 'already': tensor(-0.0221),
 'also': tensor(-0.0637),
 'although': tensor(0.0992),
 'always': tensor(-0.0548),
 'am': tensor(-0.0636),
 'an': tensor(0.1376),
 'and': tensor(0.0510),
 'angrily': tensor(-0.0372),
 'angry': tensor(0.2142),
 'another': tensor(-0.0292),
 'answer': tensor(-0.1

Now, let's convert these dot products to probabilities using the softmax function:
- We have to convert our prediction into probability distribution to get P(word|harry) so that sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1
- current dot product value is any real number, sometimes called as logit
  - logit from logistic regression. Some values that are not yet converted to 0-1 or value before sigmoid function
  - every probability should be in range (0, 1) (greater than 0, smaller than 1)
  - this can be handled by taking exponential of dot product values, divided by total sum
  - This function is called **Softmax**

- Why we use exponential?
  - Because we want to make every probability in positive range while preserving the order


In [59]:
from math import exp
word_exp_dict = {}
for word, dot_value in word_dot_dict.items():
  word_exp_dict[word] = exp(dot_value)
word_exp_dict

word_prob_dict = None

In [ ]:
# Get P(potter|harry)

## 13. Efficient Matrix Operations
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

Instead of calculating dot products one by one, we can use matrix multiplication for efficiency:


In [ ]:
# get dot product result for every word in the vocabulary

# first, make vector_of_harry into matrix format

# do matrix multiplication


Let's verify that our matrix multiplication gives the same result as individual dot products:

Now let's implement the complete softmax calculation using matrix operations:


In [ ]:
# convert dot product result into exponential

In [ ]:
# get the sum of exponential


In [ ]:
# divide exponential value with sum

## 14. Creating a Probability Function

Let's create a function to calculate probabilities efficiently:

In [ ]:
def get_probs(query_vectors, entire_vectors):
  return None

# get_probs(mat_of_harry, word_vectors)

## 15. Preparing for Training

Before training our Word2Vec model, we need to split our dataset into training and testing sets:

In [ ]:
# Now we can train the word2vec

# Let's think about training pairs
index_pairs # this is our dataset. It's list of list of two integer
# two integer means a pair of neighboring words

# Training set and Test set
# To validate that our model can solve 'unseen' problems
# So we have to split the dataset before training.

# To randomly split the dataset, we will first shuffle the dataset

# random.shuffle(index_pairs) # this will shuffle the list items

In [ ]:
len(train_set), len(test_set)

## 16. Training the Word2Vec Model

Now we'll train our Word2Vec model using batched gradient descent:

In [ ]:
# making batch from train_set
# Batch is a set of training samples, that are calculated together
# And also we update the model after one single batch

## 17. Evaluating the Training

Let's visualize the training loss to see if our model is learning:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_record)

## 18. Testing the Model

Now we'll test our model on the test set:

## 19. Exploring Learned Word Relationships

Let's explore what our model has learned by finding the words most closely related to "harry":

In [ ]:
# P(potter|harry)?
